In [1]:
from qetu_sim import *

from qiskit import *
from qiskit.assembler.disassemble import disassemble
from qiskit.qobj import QasmQobj

import json

In [11]:
degree = 30
trotter_steps = 1
u = 1
t = 1

H_ref = ref_fh_hamiltonian(u=u, t=t, WMI_qubit_layout=True, include_aux=True)
λ, v = np.linalg.eigh(H_ref)
ground_state_energy = λ[0]
ground_state_vector = v[:,0]

In [12]:
sim_ideal = AerSimulator()
simulator = sim_ideal

E_min, E_mu_m, E_mu_p, E_max = calculate_qsp_params(u, t)
qsp = QSPPhase()
phi_seq_su2 = qsp.cvx_qsp_heaviside(
    degree,
    E_min,
    E_mu_m, 
    E_mu_p, 
    E_max
)
#phi_vec = convert_Zrot_to_Xrot(phi_seq_su2)
phi_vec = phi_seq_su2

# Construct quantum circuit
QETU_circ = construct_QETU_circ(u, t, trotter_steps, phi_vec)
QETU_circ_WMI = transpile_QETU_to_WMI(QETU_circ)
onsite_circ_WMI, hop_1_circ_WMI, hop_2_circ_WMI = create_energy_estimation_circuits(QETU_circ_WMI)       


starting matlab engine..

stopping matlab engine..


In [13]:
onsite_qobj = assemble(onsite_circ_WMI)
f = open("onsite_circ.qobj", "w+")
f.write(json.dumps(onsite_qobj.to_dict(), cls=QobjEncoder))
f.close()

hop_1_qobj = assemble(hop_1_circ_WMI)
f = open("hop_1_circ.qobj", "w+")
f.write(json.dumps(hop_1_qobj.to_dict(), cls=QobjEncoder))
f.close()

hop_2_qobj = assemble(hop_2_circ_WMI)
f = open("hop_2_circ.qobj", "w+")
f.write(json.dumps(hop_2_qobj.to_dict(), cls=QobjEncoder))
f.close()

In [14]:
onsite_circ_WMI = add_pswap_labels(onsite_circ_WMI)
onsite_circ_WMI = add_sy_labels(onsite_circ_WMI)

hop_1_circ_WMI = add_pswap_labels(hop_1_circ_WMI)
hop_1_circ_WMI = add_sy_labels(hop_1_circ_WMI)

hop_2_circ_WMI = add_pswap_labels(hop_2_circ_WMI)
hop_2_circ_WMI = add_sy_labels(hop_2_circ_WMI)

In [15]:
# estimate ground state
E0_meas = estimate_ground_state_energy_from_circ(onsite_circ_WMI, hop_1_circ_WMI, hop_2_circ_WMI, u=1, t=1, num_shots=10_000)
print("E0_meas: " + str(E0_meas))
print("|E0 - E0_meas|: " + str(abs(ground_state_energy - E0_meas))) 

E0_meas: -22.61919894959143
|E0 - E0_meas|: 0.024792877235348953


In [16]:
with open('onsite_circ.qobj') as fd:
    qobj_dict = json.load(fd)
qobj = QasmQobj.from_dict(qobj_dict)
circuits, run_config, headers = disassemble(qobj)
onsite_circuit = circuits[0]

with open('hop_1_circ.qobj') as fd:
    qobj_dict = json.load(fd)
qobj = QasmQobj.from_dict(qobj_dict)
circuits, run_config, headers = disassemble(qobj)
hop_1_circuit = circuits[0]

with open('hop_2_circ.qobj') as fd:
    qobj_dict = json.load(fd)
qobj = QasmQobj.from_dict(qobj_dict)
circuits, run_config, headers = disassemble(qobj)
hop_2_circuit = circuits[0]

In [17]:
onsite_circuit = add_pswap_labels(onsite_circuit)
onsite_circuit = add_sy_labels(onsite_circuit)   

hop_1_circuit = add_pswap_labels(hop_1_circuit)
hop_1_circuit = add_sy_labels(hop_1_circuit)   

hop_2_circuit = add_pswap_labels(hop_2_circuit)
hop_2_circuit = add_sy_labels(hop_2_circuit)   

In [18]:
E0_meas = estimate_ground_state_energy_from_circ(onsite_circuit, hop_1_circuit, hop_2_circuit, u=1, t=1, num_shots=10_000)
print("E0_meas: " + str(E0_meas))
print("|E0 - E0_meas|: " + str(abs(ground_state_energy - E0_meas)))

E0_meas: -22.708450219106737
|E0 - E0_meas|: 0.06445839227995975


In [23]:
print("circuit depth: \t\t\t" + str(hop_2_circuit.depth()))
print("circuit depth w/o Z-gates: \t" + str(compute_depth_no_rz(hop_2_circuit)))

circuit depth: 			1916
circuit depth w/o Z-gates: 	1304
